In [1]:
import numpy as np
import ot
import torch
from math import sqrt
import timeit

from TheGAN import LevyGAN
from aux_functions import *

In [6]:
lines = ["abc", 'def', "ghi"]

with open("blublek.txt", "a+") as f:
    f.writelines(lines)

In [3]:
# PROOF THAT CHEN RELATION WORKS

w_dim = 4
a_dim = 6
#shit_samples = torch.randn((262144,10), dtype=torch.float)
#low_prec_samples = torch.tensor(np.genfromtxt('samples/low_prec_samples.csv', dtype=float,delimiter=','))
#mid_prec_samples = np.genfromtxt('samples/mid_prec_fixed_samples.csv', dtype=float,delimiter=',')
high_prec_samples = np.genfromtxt('samples/high_prec_fixed_samples.csv', dtype=float,delimiter=',')
#print(mid_prec_samples.shape)
print(high_prec_samples.shape)


(1048576, 10)


# 4D joint error convergence wrt bsz

In [4]:
w_dim = 4
a_dim = 6
high_prec_samples = np.genfromtxt(f'samples/fixed_samples_{w_dim}-dim.csv', dtype=float,delimiter=',')

bsz = 1
for i in range(14):
    start_time = timeit.default_timer()
    a = high_prec_samples[:bsz, w_dim:(w_dim + a_dim)]
    b = high_prec_samples[bsz:(2*bsz), w_dim:(w_dim + a_dim)]
    joint_err_high = joint_wass_dist(a, b)
    elapsed = timeit.default_timer() - start_time
    print(f"batch size: {bsz}, joint error: {joint_err_high:.4f}, time to compute: {elapsed:.4f}")
    bsz *=2

batch size: 1, joint error: 1.7390, time to compute: 0.0005
batch size: 2, joint error: 1.1340, time to compute: 0.0004
batch size: 4, joint error: 0.9144, time to compute: 0.0004
batch size: 8, joint error: 1.1132, time to compute: 0.0004
batch size: 16, joint error: 0.8568, time to compute: 0.0005
batch size: 32, joint error: 0.8304, time to compute: 0.0008
batch size: 64, joint error: 0.7002, time to compute: 0.0016
batch size: 128, joint error: 0.6450, time to compute: 0.0048
batch size: 256, joint error: 0.6025, time to compute: 0.0187
batch size: 512, joint error: 0.5312, time to compute: 0.0845
batch size: 1024, joint error: 0.4820, time to compute: 0.3942
batch size: 2048, joint error: 0.4272, time to compute: 1.7799
batch size: 4096, joint error: 0.3816, time to compute: 7.6241
batch size: 8192, joint error: 0.3414, time to compute: 31.1066


# 3D joint error convergence wrt bsz

In [5]:
w_dim = 3
a_dim = 3
high_prec_samples = np.genfromtxt(f'samples/fixed_samples_{w_dim}-dim.csv', dtype=float,delimiter=',')
bsz = 1
for i in range(14):
    start_time = timeit.default_timer()
    a = high_prec_samples[:bsz, w_dim:(w_dim + a_dim)]
    b = high_prec_samples[bsz:(2 * bsz), w_dim:(w_dim + a_dim)]
    joint_err_high = joint_wass_dist(a, b)
    elapsed = timeit.default_timer() - start_time
    print(f"batch size: {bsz}, joint error: {joint_err_high:.4f}, time to compute: {elapsed:.4f}")
    bsz *= 2

batch size: 1, joint error: 0.9698, time to compute: 0.0004
batch size: 2, joint error: 0.5247, time to compute: 0.0008
batch size: 4, joint error: 0.6439, time to compute: 0.0005
batch size: 8, joint error: 0.7396, time to compute: 0.0005
batch size: 16, joint error: 0.6804, time to compute: 0.0005
batch size: 32, joint error: 0.5724, time to compute: 0.0005
batch size: 64, joint error: 0.3949, time to compute: 0.0014
batch size: 128, joint error: 0.3260, time to compute: 0.0054
batch size: 256, joint error: 0.2690, time to compute: 0.0162
batch size: 512, joint error: 0.2300, time to compute: 0.0645
batch size: 1024, joint error: 0.1974, time to compute: 0.3165
batch size: 2048, joint error: 0.1687, time to compute: 1.2989
batch size: 4096, joint error: 0.1313, time to compute: 5.8474
batch size: 8192, joint error: 0.1040, time to compute: 26.9113


In [5]:
samples = mid_prec_samples
for i in range(10):
    print(samples.shape)
    err = [sqrt(ot.wasserstein_1d(samples[:,w_dim+i],high_prec_samples[:,w_dim+i],p=2)) for i in range(a_dim)]
    pretty_err = ["{0:0.5f}".format(i) for i in err]
    print(pretty_err)
    samples = chen_combine(samples, w_dim)

torch.Size([524288, 10])
['0.11013', '0.11028', '0.11219', '0.11458', '0.11457', '0.11050']
torch.Size([262144, 10])
['0.05057', '0.04843', '0.05067', '0.05358', '0.05318', '0.04986']
torch.Size([131072, 10])
['0.02475', '0.02271', '0.02295', '0.02824', '0.02522', '0.02398']
torch.Size([65536, 10])
['0.01272', '0.01225', '0.01122', '0.01766', '0.01311', '0.01301']
torch.Size([32768, 10])
['0.01055', '0.01206', '0.01073', '0.01384', '0.01109', '0.01370']
torch.Size([16384, 10])
['0.01178', '0.01128', '0.01265', '0.01510', '0.01129', '0.01640']
torch.Size([8192, 10])
['0.01348', '0.01753', '0.02108', '0.01981', '0.01925', '0.02471']
torch.Size([4096, 10])
['0.02155', '0.01392', '0.03072', '0.02521', '0.01897', '0.04090']
torch.Size([2048, 10])
['0.01774', '0.03060', '0.04047', '0.02391', '0.03276', '0.03484']
torch.Size([1024, 10])
['0.03906', '0.03272', '0.04108', '0.03886', '0.03452', '0.04955']


In [3]:
samples = torch.tensor(gen_2mom_approx(w_dim,1048576), dtype=torch.float)
for i in range(10):
    print(samples.shape)
    err = [sqrt(ot.wasserstein_1d(samples[:,w_dim+i],high_prec_samples[:,w_dim+i],p=2)) for i in range(a_dim)]
    pretty_err = ["{0:0.5f}".format(i) for i in err]
    print(pretty_err)
    samples = chen_combine(samples, w_dim)

torch.Size([1048576, 10])
['0.02043', '0.02028', '0.02015', '0.02070', '0.02021', '0.02093']
torch.Size([524288, 10])
['0.00469', '0.00463', '0.00376', '0.00489', '0.00364', '0.00491']
torch.Size([262144, 10])
['0.00326', '0.00332', '0.00398', '0.00399', '0.00261', '0.00406']
torch.Size([131072, 10])
['0.00328', '0.00498', '0.00557', '0.00441', '0.00431', '0.00431']
torch.Size([65536, 10])
['0.00752', '0.00591', '0.00738', '0.00606', '0.00560', '0.00498']
torch.Size([32768, 10])
['0.00778', '0.00933', '0.00952', '0.00857', '0.00725', '0.00681']
torch.Size([16384, 10])
['0.01025', '0.00973', '0.01364', '0.01684', '0.01440', '0.01251']
torch.Size([8192, 10])
['0.01143', '0.01203', '0.01242', '0.02114', '0.01369', '0.01242']
torch.Size([4096, 10])
['0.02291', '0.01600', '0.02695', '0.01790', '0.02083', '0.01911']
torch.Size([2048, 10])
['0.03958', '0.02357', '0.03886', '0.03233', '0.04769', '0.04406']


In [6]:
samples = torch.randn((262144,10), dtype=torch.float)
for i in range(10):
    print(samples.shape)
    err = [sqrt(ot.wasserstein_1d(samples[:,w_dim+i],high_prec_samples[:,w_dim+i],p=2)) for i in range(a_dim)]
    pretty_err = ["{0:0.5f}".format(i) for i in err]
    print(pretty_err)
    samples = chen_combine(samples, w_dim)

torch.Size([262144, 10])
['0.51340', '0.51083', '0.51092', '0.50750', '0.50810', '0.51202']
torch.Size([131072, 10])
['0.30308', '0.30325', '0.29886', '0.29845', '0.30054', '0.30077']
torch.Size([65536, 10])
['0.17498', '0.17328', '0.16919', '0.16641', '0.17082', '0.17315']
torch.Size([32768, 10])
['0.09511', '0.09724', '0.09499', '0.08869', '0.09437', '0.09759']
torch.Size([16384, 10])
['0.05233', '0.05037', '0.04912', '0.04466', '0.05436', '0.05230']
torch.Size([8192, 10])
['0.02287', '0.02607', '0.02541', '0.03164', '0.03131', '0.03569']
torch.Size([4096, 10])
['0.02542', '0.02252', '0.01838', '0.02700', '0.02225', '0.03080']
torch.Size([2048, 10])
['0.02568', '0.03141', '0.02792', '0.03268', '0.03277', '0.04789']
torch.Size([1024, 10])
['0.03117', '0.04250', '0.04890', '0.03724', '0.04217', '0.03931']
torch.Size([512, 10])
['0.04673', '0.06918', '0.04965', '0.08538', '0.06252', '0.06134']


In [4]:
# Calculate E[ A^2 | W ] = 1/12 * (1 + (W_1)^2 + (W_2)^2)

def true_second_moments(_w):
    _w_dim = len(_w)
    _w_squared = [x*x for x in _w]
    _a_dim = int((_w_dim - 1)*_w_dim/2)
    moms = []
    for k in range(_w_dim):
        for l in range(k+1,_w_dim):
            moms.append((1.0/12.0)*(1.0 + _w_squared[k] + _w_squared[l]))
    assert len(moms) == _a_dim
    return np.diag(moms)


def empirical_second_moments(_a_generated: np.ndarray):

    _batch_dim = _a_generated.shape[0]
    const = 1.0/_batch_dim
    _a_dim = _a_generated.shape[1]
    result = np.zeros((_a_dim,_a_dim))
    for i in range(_a_dim):
        for j in range(i, _a_dim):
            result[i,j] = const * np.dot(_a_generated[:,i], _a_generated[:,j])
            if i != j:
                result[j,i] = result[i,j]

    return result



def avg_st_dev_error(_w, _a_generated):
    difference = np.abs(np.sqrt(true_second_moments(_w)) - np.sqrt(np.abs(empirical_second_moments(_a_generated))))
    return difference.mean()

def empirical_variances(_a_generated):
    return np.diagonal(empirical_second_moments(_a_generated))

In [5]:
w_dim = 5
a_dim = 10
w_fixed = [1.0,-0.5,-1.2,-0.3,0.7]

# test_filename = f"samples/fixed_samples_{w_dim}-dim.csv"
# A_fixed_true = np.genfromtxt(test_filename,dtype=float,delimiter=',',)
# A_fixed_true = A_fixed_true[:,w_dim:(w_dim+a_dim)]
A_fixed_true = np.zeros((10,10))



a = true_second_moments(w_fixed)
b = empirical_second_moments(A_fixed_true)
c = empirical_variances(A_fixed_true)
print(np.diagonal(a))
# print(b)
print(c)
print(avg_st_dev_error(w_fixed,A_fixed_true))

[0.1875     0.28666667 0.17416667 0.2075     0.22416667 0.11166667
 0.145      0.21083333 0.24416667 0.13166667]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.043458536658709475


In [12]:
filename = f'model_saves/blubiran.txt'

with open(filename, 'a+') as summary_file:
    summary_file.seek(0)
    lines = summary_file.read().splitlines()
    if lines == []:
        serial_num = 1
        summary_file.write(str(serial_num))
    else:
        last_line = lines[-1]
        serial_num = int(last_line)

    summary = f" descriptor: " + "report" + f"\n{serial_num + 1}"
    summary_file.write(summary)



In [9]:
beta1 = 0.2
beta2 = 0.95
print(f"beta1_{beta1:.2f}_beta2_{beta2:.3f}")

beta1_0.20_beta2_0.950
